# AUC Analysis: Alulose vs Fructose

This notebook calculates the Area Under the Curve (AUC) for glucose response data comparing alulose and fructose, using two methods:

1. **Standard Method**: Uses trapezoidal integration with error propagation
   - Calculates AUC using trapezoidal rule on mean values
   - Estimates uncertainty using two approaches and takes the more conservative:
     a. Normal distribution assumption: std = (upper_bound - lower_bound) / 6
     b. Error propagation through trapezoidal rule
   
2. **Monte Carlo Method**: Uses simulation-based estimation
   - Samples 10,000 curves from normal distributions defined by means and standard deviations
   - Calculates AUC for each sampled curve
   - Reports mean and standard deviation of the resulting AUCs

In [ ]:
import pandas as pd
import numpy as np

# Define the number of participants (used for SEM to SD conversion)
n_participants = 25

data = {
    "Mean X": [0, 30, 60, 90, 120],
    "Mean Y": [0, 3.268958606, 2.480386944, 1.403795812, 0.785319863],
    "Lower X": [0, 30, 60, 90, 120],
    "Lower Y": [0, 3.268958606, 2.480386944, 1.142015707, 0.523539758],
    "Upper X": [0, 30, 60, 90, 120],
    "Upper Y": [0, 3.268958606, 2.873016198, 1.403795812, 1.295791067],
}

df_alulose = pd.DataFrame(data)

data2 = {
    "Mean X": [0, 30, 60, 90, 120],
    "Mean Y": [0, 3.412978567, 2.676660668, 1.940404123, 1.125633999],
    "Lower X": [0, 30, 60, 90, 120],
    "Lower Y": [0, 3.412978567, 2.676660668, 1.691733475, 0.510450753],
    "Upper X": [0, 30, 60, 90, 120],
    "Upper Y": [0, 3.648580661, 3.00390625, 2.189115674, 1.361256545],
}

df_fructose = pd.DataFrame(data2)

# Calculate standard deviations from SEM
for df in [df_alulose, df_fructose]:
    # Calculate SEM as average of upper and lower differences
    upper_diff = df['Upper Y'] - df['Mean Y']
    lower_diff = df['Mean Y'] - df['Lower Y']
    sem = np.maximum(upper_diff, lower_diff)
    
    # Convert SEM to standard deviation
    df['std'] = sem * np.sqrt(n_participants)

In [ ]:
import matplotlib.pyplot as plt

df = df_alulose
plt.errorbar(df["Mean X"], df["Mean Y"], yerr=[df["Mean Y"] - df["Lower Y"], df["Upper Y"] - df["Mean Y"]], label="10g allulose", fmt='o--', color="blue", ecolor="blue", elinewidth=2, capsize=4)
df = df_fructose
plt.errorbar(df["Mean X"], df["Mean Y"], yerr=[df["Mean Y"] - df["Lower Y"], df["Upper Y"] - df["Mean Y"]], label="10g fructose", fmt='o--', color="orange", ecolor="orange", elinewidth=2, capsize=4)

plt.xlabel("Time (mins)")
plt.ylabel("∆ Plasma Glucose (mmol/L)")
plt.legend()
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.savefig("alulose_vs_fructose.pdf", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
from auc_tools import standard_auc, monte_carlo_auc
import pandas as pd

# Calculate the standard deviation for both dataframes
df_alulose['std'] = df_alulose[['Mean Y']].sub(df_alulose['Lower Y'], axis=0).max(axis=1).combine(
    df_alulose['Upper Y'].sub(df_alulose['Mean Y']), max
) + 0.0001
df_fructose['std'] = df_fructose[['Mean Y']].sub(df_fructose['Lower Y'], axis=0).max(axis=1).combine(
    df_fructose['Upper Y'].sub(df_fructose['Mean Y']), max
) + 0.0001

# Initialize an empty list to store results
results = []

# Loop through both dataframes and calculate AUC metrics
for df, label in zip([df_alulose, df_fructose], ["alulose", "fructose"]):
    standard_mean, lower, upper = standard_auc(df['Mean X'], df['Mean Y'], df['std'])
    standard_std = (upper - lower) / 2
    mc_mean, mc_std = monte_carlo_auc(df['Mean X'].to_list(), df['Mean Y'].to_list(), df['std'].to_list())
    
    # Append results as a dictionary
    results.append({
        "Label": label,
        "Standard AUC Mean": standard_mean,
        "Standard AUC Std": standard_std,
        "Monte Carlo AUC Mean": mc_mean,
        "Monte Carlo AUC Std": mc_std
    })

# Convert the results list into a DataFrame
results_df = pd.DataFrame(results)

# Display the results
results_df.to_csv("auc_example_results.csv", index=False)
results_df